In [1]:
%load_ext autoreload
%autoreload 2


In [41]:
from flair.models import SequenceTagger
from dkoulinker.entity_linker import EntityLinker, get_mentions_ner, get_mentions_flair, proc_flair_sentence
from dkoulinker.tokenization import SegtokSentenceSplitter
from flair.data import Sentence


In [3]:
# load the NER tagger
tagger = SequenceTagger.load(
    '../resources/taggers/sota-ner-flair/best-model.pt')


2021-12-14 15:22:44,815 loading file ../resources/taggers/sota-ner-flair/best-model.pt


In [4]:
text = "Zika virus is a mosquitoborne flavivirus that is the focus of an ongoing pandemic and public health emergency. Previously limited to sporadic cases in Africa and Asia, the emergence of Zika virus in Brazil in 2015 heralded rapid spread throughout the Americas. Although most Zika virus infections are characterized by subclinical or mild influenza-like illness, severe manifestations have been described, including Guillain-Barre syndrome in adults and microcephaly in babies born to infected mothers. Neither an effective treatment nor a vaccine is available for Zika virus; therefore, the public health response primarily focuses on preventing infection, particularly in pregnant women. Despite growing knowledge about this virus, questions remain regarding the virus's vectors and reservoirs, pathogenesis, genetic diversity, and potential synergistic effects of co-infection with other circulating viruses. These questions highlight the need for research to optimize surveillance, patient management, and public health intervention in the current Zika virus epidemic. "
get_mentions_ner(text, tagger)


<function dkoulinker.entity_linker.proc_flair_sentence(sentence)>

In [5]:
# from flair.tokenization import SegtokSentenceSplitter


In [42]:
%%time
# example text with many sentences
text = "Zika virus is a mosquitoborne flavivirus that is the focus of an ongoing pandemic and public health emergency. Previously limited to sporadic cases in Africa and Asia, the emergence of Zika virus in Brazil in 2015 heralded rapid spread throughout the Americas. Although most Zika virus infections are characterized by subclinical or mild influenza-like illness, severe manifestations have been described, including Guillain-Barre syndrome in adults and microcephaly in babies born to infected mothers. Neither an effective treatment nor a vaccine is available for Zika virus; therefore, the public health response primarily focuses on preventing infection, particularly in pregnant women. Despite growing knowledge about this virus, questions remain regarding the virus's vectors and reservoirs, pathogenesis, genetic diversity, and potential synergistic effects of co-infection with other circulating viruses. These questions highlight the need for research to optimize surveillance, patient management, and public health intervention in the current Zika virus epidemic. "

# initialize sentence splitter
splitter = SegtokSentenceSplitter()

# use splitter to split text into list of sentences
plain_sentences, sentences = splitter.split(text)

# predict tags for sentences
# tagger = SequenceTagger.load('ner')
tagger.predict(sentences)

# iterate through sentences and print predicted labels
# for sentence in sentences:
#     print(sentence.to_tagged_string())


2021-12-14 16:29:20,520 Warning: An empty Sentence was created! Are there empty strings in your dataset?
2021-12-14 16:29:20,521 Ignore 1 sentence(s) with no tokens.
CPU times: user 38.1 ms, sys: 10 ms, total: 48.2 ms
Wall time: 47.1 ms


In [44]:
ner_mentions = []  # a list of dicts
last_sen_size = 0
for i,sentence in enumerate(sentences):
    # if len(sentence) == 0:
    #     continue
    # print(len(sentence))
    # print(sentence.to_tagged_string())
    ner_mentions_textonly_sentence, ner_mentions_sentence = proc_flair_sentence(
        sentence)
    if len(ner_mentions_sentence) > 0:
        for ment in ner_mentions_sentence:

            ment['start_pos'] += last_sen_size
            ment['end_pos'] += last_sen_size

        #Skip mentions not in dictionary
            # if ment['text'] in self.mention2pem:
            ner_mentions.append(ment)
#Last text lenght plus 1 for accounting the '.'
    last_sen_size += len(plain_sentences[i]) + 1


In [45]:
ner_mentions

[{'text': 'flavivirus', 'start_pos': 30, 'end_pos': 40},
 {'text': 'treatment', 'start_pos': 523, 'end_pos': 532}]

In [38]:
%%time
sent = Sentence(text)
tagger.predict(sent)

CPU times: user 110 ms, sys: 60.7 ms, total: 171 ms
Wall time: 170 ms


In [39]:
sent.to_tagged_string

<bound method Sentence.to_tagged_string of Sentence: "Zika virus is a mosquitoborne flavivirus that is the focus of an ongoing pandemic and public health emergency . Previously limited to sporadic cases in Africa and Asia , the emergence of Zika virus in Brazil in 2015 heralded rapid spread throughout the Americas . Although most Zika virus infections are characterized by subclinical or mild influenza-like illness , severe manifestations have been described , including Guillain-Barre syndrome in adults and microcephaly in babies born to infected mothers . Neither an effective treatment nor a vaccine is available for Zika virus ; therefore , the public health response primarily focuses on preventing infection , particularly in pregnant women . Despite growing knowledge about this virus , questions remain regarding the virus 's vectors and reservoirs , pathogenesis , genetic diversity , and potential synergistic effects of co-infection with other circulating viruses . These questions hig

In [24]:
%%time
sentences = [Sentence(sent) for sent in text.split('.')]
for sentence in sentences:
    # print(len(sentence))
    tagger.predict(sentences)
    print(sentence.to_plain_string())


2021-12-14 15:56:49,876 Ignore 1 sentence(s) with no tokens.
Zika virus is a mosquitoborne flavivirus that is the focus of an ongoing pandemic and public health emergency
2021-12-14 15:56:49,919 Ignore 1 sentence(s) with no tokens.
Previously limited to sporadic cases in Africa and Asia, the emergence of Zika virus in Brazil in 2015 heralded rapid spread throughout the Americas
2021-12-14 15:56:49,961 Ignore 1 sentence(s) with no tokens.
Although most Zika virus infections are characterized by subclinical or mild influenza-like illness, severe manifestations have been described, including Guillain-Barre syndrome in adults and microcephaly in babies born to infected mothers
2021-12-14 15:56:50,003 Ignore 1 sentence(s) with no tokens.
Neither an effective treatment nor a vaccine is available for Zika virus; therefore, the public health response primarily focuses on preventing infection, particularly in pregnant women
2021-12-14 15:56:50,046 Ignore 1 sentence(s) with no tokens.
Despite gr